In [30]:
import pandas as pd
import requests
from io import StringIO

# %conda install lxml
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win 64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
# Mozilla/5.0 (Windows NT 10.0; Win 64; x64) 
# AppleWebKit/537.36 (KHTML, like Gecko) 
# Chrome/91.0.4472.124 Safari/537.36

url = "https://en.wikipedia.org/wiki/List_of_footballers_with_500_or_more_goals"
# https://en.wikipedia.org/wiki/
# List_of_footballers_with_500_or_more_goals
res = requests.get(url, headers=headers).text
res = StringIO(res)

df = pd.read_html(res)
df


[    0                                                  1
 0 NaN  This section needs additional citations for ve...,
    Rank              Player    Club                  Country and other  \
    Rank              Player  League  Cup Continental Country and other   
 0     1   Cristiano Ronaldo  590[a]   57         172               143   
 1     2       Lionel Messi*  553[b]   71         157               115   
 2     3               Pelé*  604[c]   49          26                83   
 3     4             Romário  545[d]   93          54                64   
 4     5       Ferenc Puskás  516[e]   69          56                84   
 5     6        Josef Bican*  515[f]  137          38                32   
 6     7  Robert Lewandowski  423[g]   62         117                88   
 7     8        Jimmy Jones*  330[h]  286          14                 9   
 8     9        Gerd Müller*  405[i]   92          69                68   
 9    10       Joe Bambrick*  347[j]  253           5     

In [31]:
df = df[3]

In [32]:
# df.to_csv("./data/top500goals.csv", index=False)

In [33]:
df = pd.read_csv("./data/top500goals.csv")
df.head()

,Rank,Player,Goals,Matches,Ratio,Career span
0,1,Erwin Helmchen,989+,582,1.70,1924–1951
1,2,Cristiano Ronaldo,975,1337,0.73,2002–present
2,3,Josef Bican,950+,624,1.52,1930–1957
3,4,Ronnie Rooke,934+,1030,0.91,1929–1961
4,5,Lionel Messi,925,1194,0.77,2003–present


In [34]:
import random, math

df["Career span"] = df["Career span"].str.split("–").str[0]

In [35]:
df["Career span"].isna().sum()

np.int64(0)

In [36]:
# df.loc[~df['Goals'].str.isnumeric()]
df["Goals"] = [item.replace("+", "") for item in df["Goals"]]
df["Goals"]

0     989
1     975
2     950
3     934
4     925
     ... 
78    504
79    504
80    503
81    502
82    500
Name: Goals, Length: 83, dtype: object

In [37]:
df["Goals"] = df["Goals"].astype(int)
df["Career span"] = df["Career span"].astype(int)
df.head()

,Rank,Player,Goals,Matches,Ratio,Career span
0,1,Erwin Helmchen,989,582,1.70,1924
1,2,Cristiano Ronaldo,975,1337,0.73,2002
2,3,Josef Bican,950,624,1.52,1930
3,4,Ronnie Rooke,934,1030,0.91,1929
4,5,Lionel Messi,925,1194,0.77,2003


In [38]:
df["Matches"] = df["Matches"].str.replace("+", "").astype(int)
df.describe()

,Rank,Goals,Matches,Ratio,Career span
count,83.000000,83.000000,83.000000,83.000000,83.00000
mean,42.000000,606.361446,724.915663,0.886024,1945.53012
std,24.103942,122.085394,193.411996,0.267563,26.39306
min,1.000000,500.000000,324.000000,0.530000,1891.00000
25%,21.500000,521.000000,590.500000,0.695000,1927.50000
50%,42.000000,564.000000,711.000000,0.860000,1938.00000
75%,62.500000,631.000000,836.500000,0.960000,1961.00000
max,83.000000,989.000000,1337.000000,1.700000,2010.00000


In [39]:
from datetime import datetime, timedelta
rows = []
columns = ["name", "date", "goals", "assists"]

for row in df.itertuples():
    name = row.Player
    start_date = f"{row._6}-08-01"
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    goal_ratio = row.Ratio
    goal_sd = random.random()
    goal_sd = goal_sd if goal_sd < 0.7 else 0.7
    assist_sd = 1 - goal_sd
    assist_ratio = 1- goal_ratio    
    
    for i in range(row.Matches):
        name = name
        match_date = timedelta(days=3 * i) + start_date
        goals = random.normalvariate(goal_ratio, goal_sd)
        assists = random.normalvariate(assist_ratio, assist_sd)
        goals = round(goals) if goals > 0 else 0
        assists = round(assists) if assists > 0 else 0
        row = [name, match_date, goals, assists]
        rows.append(row)
games_df = pd.DataFrame(data=rows, columns=columns)
games_df.head()
        

,name,date,goals,assists
0,Erwin Helmchen,1924-08-01,2,1
1,Erwin Helmchen,1924-08-04,1,0
2,Erwin Helmchen,1924-08-07,2,0
3,Erwin Helmchen,1924-08-10,1,0
4,Erwin Helmchen,1924-08-13,2,0


In [41]:
games_df.shape

(60168, 4)

In [43]:
games_df.sample(10)

,name,date,goals,assists
10509,Robert Lewandowski,2005-09-25,2,0
6572,Ferenc Deák,1941-08-29,1,0
39467,Jock Dodds,1933-10-19,1,1
44380,Nándor Hidegkuti,1941-01-23,1,0
8227,Romário,1989-01-16,0,0
1023,Cristiano Ronaldo,2006-03-16,1,0
58498,Albert de Cleyn,1935-12-28,0,0
20288,Luis Suárez,2006-04-25,1,2
13473,Joe Bambrick,1929-07-11,1,0
59372,Delio Onnis,1971-05-04,1,1


In [ ]:
fake_df = games_df.groupby("name").agg(
    matches=("name", "count"),
    goals=("goals", "sum"),
    assists=("assists", "sum"),
    g_ratio=("goals", "mean"),
    a_ratio=("assists", "mean"),
).reset_index().sort_values(by="goals", ascending=False).reset_index(drop=True)
# fake_df["goal_ratio"] = fake_df["goals"] / fake_df["matches"]
fake_df.head(10)

,name,matches,goals,assists,g_ratio,a_ratio
0,Erwin Helmchen,582,1122,41,1.927835,0.070447
1,Cristiano Ronaldo,1337,1014,479,0.758414,0.358265
2,Lionel Messi,1194,992,222,0.830821,0.185930
3,Josef Bican,624,952,0,1.525641,0.000000
4,Ferenc Bene,972,947,572,0.974280,0.588477
5,Ronnie Rooke,1030,940,288,0.912621,0.279612
6,Pelé,851,844,315,0.991774,0.370153
7,Jimmy Greaves,842,819,418,0.972684,0.496437
8,Ferenc Deák,515,809,0,1.570874,0.000000
9,Jimmy Jones,760,798,164,1.050000,0.215789


In [49]:
for i in range(10):
    print(f"{i} df: {df.loc[i,["Player", "Goals"]]} | {i} fake: {fake_df.loc[i,["name", "goals"]]}")

0 df: Player    Erwin Helmchen
Goals                989
Name: 0, dtype: object | 0 fake: name     Erwin Helmchen
goals              1122
Name: 0, dtype: object
1 df: Player    Cristiano Ronaldo
Goals                   975
Name: 1, dtype: object | 1 fake: name     Cristiano Ronaldo
goals                 1014
Name: 1, dtype: object
2 df: Player    Josef Bican
Goals             950
Name: 2, dtype: object | 2 fake: name     Lionel Messi
goals             992
Name: 2, dtype: object
3 df: Player    Ronnie Rooke
Goals              934
Name: 3, dtype: object | 3 fake: name     Josef Bican
goals            952
Name: 3, dtype: object
4 df: Player    Lionel Messi
Goals              925
Name: 4, dtype: object | 4 fake: name     Ferenc Bene
goals            947
Name: 4, dtype: object
5 df: Player    Jimmy Jones
Goals             840
Name: 5, dtype: object | 5 fake: name     Ronnie Rooke
goals             940
Name: 5, dtype: object
6 df: Player    Ferenc Puskás
Goals               802
Name: 6, dtype